In [34]:
from PIL import Image
import numpy as np
from numpy import genfromtxt
from sklearn.decomposition import PCA, TruncatedSVD
import random
from scipy.stats import multivariate_normal
#ORDEN DE REVISIÓN SUGERIDO: 
# 1) kmeans()
# 2) load()
# 3) iteration()
# 4) distance()
#EL MAIN SE ENCUENTRA AL PIE DEL ARCHIVO


def distance(a, b):
    #DISTANCIA EUCLIDIANA N DIMENSIONAL
    aux = np.sum(np.square(a - b))
    return np.sqrt(aux)


def iteration(k_puntos, puntos, dim_datos):
    #CALCULO DE DISTANCIA DE CADA DATO CON LOS K CENTROIDES DE LOS CLUSTERES Y ASIGNAMIENTO DEL CLUSTER MAS CERCANO PARA CADA DATO
    clusters = []
    for idx, punto in enumerate(puntos):
        d = 99999999999991
        ki = 0
        for idx_k, k_i in enumerate(k_puntos):
            if(d > distance(punto, k_i)):
                d = distance(punto, k_i)
                ki = idx_k
        clusters.append(ki)




    #CONTABILIZACION DE ELEMENTOS POR CADA CLUSTER Y SUMA DE FEATURES PARA CADA UNO DE ELLOS
    aux = [0] * dim_datos
    means = [np.array(aux)] * len(puntos)
    count = [0] * len(puntos)
    for i in range(0,len(puntos)):
        idx = clusters[i]
        means[idx] = np.add(puntos[i], means[idx])
        count[idx] += 1


    #CÁLCULO DE LA MEDIA PARA CADA CLUSTER Y REASIGNACIÓN DE LOS K CENTROIDES
    k_p = []
    for j in range(len(means)):
        # print(means[i])
        # print(count[i])
        # break
        if(count[j] != 0):
            #print('a')
            for i in range(len(means[j])):
                means[j][i] = means[j][i]/count[j]
            k_p.append(np.array(means[j]))
    k_puntos = k_p

    #RETORNO DE DATOS
    return k_puntos, clusters

def load(k, redux,redux_type, q):

    #PROCESAMIENTO DE DATOS
    borrarcomillas = lambda x: x.replace('"', '')
    names = np.genfromtxt('clase.csv', dtype=None, delimiter=",", skip_header=1, usecols=1, encoding='utf-8', converters={1: borrarcomillas})
    data = np.genfromtxt('dataset.csv', delimiter=",")

    #REDUCCION DE DIMENSIONALIDAD
    svd = TruncatedSVD(n_components=redux)
    pca = PCA(n_components=redux)
    datos = data
    if redux_type == 0:
        datos = svd.fit(data).transform(data)
    elif redux_type == 1:
        datos = pca.fit(data).transform(data)


    '''
    maximo = 0
    minimo = 9999999999999
    for i in range(len(datos)):
        for j in range(len(datos[i])):
            #datos[i][j] = 1.08**datos[i][j]
            if maximo < datos[i][j]:
                maximo = datos[i][j]
            if minimo > datos[i][j]:
                minimo = datos[i][j]
    '''

    #VARIABLES UTILES
    n_datos = len(datos)
    dim_datos = len(datos[0])
    
    #CARGA DE PUNTOS
    puntos = []
    k_puntos = []
    for i in range(n_datos):
        puntos.append(np.array(datos[i]))

    #OBTENCIÓN DE PROMEDIOS DE CADA FEATURE Y SU DESVIACIÓN ESTANDAR

    data_ = datos.transpose()
    sd = []
    promedios = []
    for x in data_:
        sd.append(np.std(x))
        promedios.append(np.mean(x))
    

    #GENERACION DE LOS K PUNTOS PARA LOS CLUSTERES USANDO UN RANGO DETERMINADO POR EL PROMEDIO + Q*DESVIACION ESTANDAR DONDE Q ES UN NUMERO ARBITRARIO PERO QUE TENGA SENTIDO
    np.random.seed(44)
    for i in range(k):
        aux = []
        for i in range(dim_datos):
            aux.append(np.random.randint(round(promedios[i]-q*sd[i]),round(promedios[i]+q*sd[i])))
        k_puntos.append(np.array(aux))

  
    #RETORNO DE LOS K PUNTOS PARA LOS CLUSTERES, LOS PUNTOS CARGADOS, LA DIMENSION DE LOS DATOS Y LOS LABELS CORRECTOS
    return k_puntos, puntos, dim_datos, names



def kmeans(k, epoch, redux, redux_type, q):

    #CARGA DE DATOS
    k_puntos, puntos, dim_datos, names = load(k, redux, redux_type, q)

    
    #MECANISMO PARA IMPRIMIR PORCENTAJE ACTUAL DE LA OPERACIÓN Y SABER CUANTO APROXIMADAMENTE FALTA PARA QUE TERMINE EL ALGORITMO
    suma = 0
    if(epoch >= 100):
        per = epoch / 100
        suma = 1
    else:
        per = 100/epoch
        suma = per
        
    tiempo = 0
    count = 0


    #LLAMADA AL ALGORITMO iteration() CON ITERACIONES DE REFINAMIENTO=EPOCH E IMPRESIÓN DE PORCENTAJE ACTUAL DE EJECUCIÓN DEL ALGORITMO
    clusters = []
    for i in range(epoch):
        if tiempo >= per:
            # print(str(count)+'%')
            count += suma
            tiempo = 0
        k_puntos,clusters = iteration(k_puntos, puntos, dim_datos)
        tiempo+=suma
    # print(str(100)+'%')  
            
    
    #FORMATEO CORRECTO DEL RESULTADO PARA DISPLAY 
    results = []
    objects = []



    cluster_map = {}
    cluster_features_means = {}
    for i in range(k):
        aux = []
        elements_in_cluster = []
        for j in range(len(clusters)):
            if clusters[j] == i:
                aux.append(puntos[j])
                elements_in_cluster.append(puntos[j])
        
        #estoy en el cluster i con elements_in_clusters elementos en dicho cluster
        features_prom = []
        for x in range(dim_datos):
            prom = 0
            for y in range(len(elements_in_cluster)):
                prom += elements_in_cluster[y][x]

            prom = prom / dim_datos

            features_prom.append(prom)


        cluster_map[i] = np.array(aux)
        cluster_features_means[i] = np.array(features_prom)

    for i in range(k):
        aux = []
        aux1 = []
        for j in range(len(clusters)):
            if clusters[j] == i:
                aux.append(names[j])
                
        results.append(aux)
    
    #IMPRESIÓN DE RESULTADOS
    count = 0
    for i in results:
        print('Cluster:', count, '\n')
        print(i)
        count +=1

    #RETORNO DE RESULTADOS

    return clusters, cluster_map, cluster_features_means, np.array(puntos), names




#MAIN
# k = cantidad de clusteres
# epochs = cantidad de iteraciones de 'refinación'
# redux = cantidad de dimensiones finales que tendrá el dataset (pca puede quejarse si le pones mas de N, siendo N la cantidad de datos)
# redux_type 0 = svd, 1 = pca
# q = cantidad de veces que se suma y resta la desviación estandar a la media para obtener un rango de generación para las features de los centroides aleatorios. 
# rango para cada feature = [ft_mean-q*ft_sd, ft_mean+q*ft_sd]

# kmeans() retorna un arreglo arr[] en donde cada posición i representa el elemento i del dataset[] ya reducido y procesado, y arr[i] contiene el número de cluster que se le ha asignado. El arreglo names[i] contiene el label real del elemento dataset[i]
# la reducción de dimensionalidad se hace en la función load(), dejaré un indicador para que sea evidente


#print(cm[0])
# print(cfm[0])
#PROBLEMAS ENCONTRADOS:

#CLUSTERS CON DOS CATEGORÍAS
#Posible respuesta: Algunas categorías de labels no aparecen en un cluster propio, puede deberse a que la naturaleza de los features de dicha categoría sean demasiado similares a otra categoría, por lo que se clusteriza en el mismo sitio o que en la reducción de dimensionalidad, se hayan acortado diferencias entre dos categorías y sus labels, lo que resulta en la primera observación.

#MAS DE UN CLUSTER PARA UNA CATEGORÍA
#Posible respuesta: Los features de los datos de una misma categoría tienen tendencia a diverger, con la reducción de dimensionalidad, en dos o más grupos diferentes o (la que creo que es) dos centroides aleatorios a1 y a2 cayeron muy cerca entre sí y se formaron dos miniclusters que representan lo mismo. NOTA: como el rango de generación de cada feature para los centroides aleatorios es media+-q*desviacion estandar, si sucede la primera observación, quiere decir que los rangos probablemente resultaron muy pequeños, por lo que se generaron puntos con features similares y habría que aumentar el q para dar mas holgura.

#CLUSTERES VACÍOS
#Posible respuesta: Se generó un centroide tan lejano que no resultó ser el más cercano para ningún dato, habría que evaluar disminuir el q para disminuir la holgura de generación y crear centroides más cercanos a los puntos.



In [42]:
def new_medias(gammas, N, n_samples, n_classes, k_clusters):
  medias = np.zeros((k_clusters, n_classes))
  for i in range(k_clusters):
    media = np.full((n_classes,), 0, dtype='float64')
    for n in range(n_samples):
      media += gammas[n,i]*dataset[n,]
    media = media/N[i]
    medias[i,] += media
  return medias

def new_covarianzas(gammas, medias, N, n_samples, n_classes, k_clusters):
  covarianzas = np.zeros((k_clusters, n_classes, n_classes))

  for i in range(k_clusters):
    covarianza = np.zeros((n_classes, n_classes))
    for n in range(n_samples):
      columna = dataset[n,] - medias[i,]
      columna = columna.reshape(n_classes, 1)
      fila = columna.reshape(1, n_classes)
      covarianza += gammas[n,i]*(columna@fila)
    covarianzas[i,:,:] += covarianza/N[i]

  return covarianzas

def new_N(gammas, k_clusters, n_samples):
  N = np.zeros((k_clusters,))
  for i in range(k_clusters):
    val = 0.0
    for n in range(n_samples):
      val += gammas[n,i]
    N[i,] += val

  return N

def new_pis(N, n_samples, k_clusters):
  pis = np.zeros((k_clusters,))
  for i in range(k_clusters):
    pis[i,] += N[i]/n_samples
  return pis

epsilon = 0.0001

def new_responsibilities(pis, medias, covarianzas, n_samples, k_clusters):
  gammas = np.zeros((n_samples, k_clusters))
  for n in range(n_samples):
    for k in range(k_clusters):
      numerador = pis[k,]*(multivariate_normal.pdf(dataset[n,], mean=medias[k,], cov=covarianzas[k,:,:], allow_singular=True)+epsilon)
      # print('muultivariate sin epsilon: ', multivariate_normal.pdf(dataset[n,], mean=medias[k,], cov=covarianzas[k,:,:], allow_singular=True))
      # print('pis[k,]: ', pis[k,])
      # print('covs: ', covarianzas[k,:,:])
      # print('medias: ', medias[k,])
      # print('numerador: ', numerador)
      denominador = 0.0
      for _k in range(k_clusters):
        denominador += pis[_k,]*(multivariate_normal.pdf(dataset[n,], mean=medias[_k,], cov=covarianzas[_k,:,:], allow_singular=True)+epsilon)
      # print('denominador: ', denominador)
      res = numerador/denominador
      gammas[n, k] += res
  return gammas

def log_likelihood(k_clusters, medias, covarianzas, n_samples, pis):
  prob = 0
  for n in range(n_samples):
    res = 0
    for k in range(k_clusters):
      res += pis[k,]*(multivariate_normal.pdf(dataset[n,], mean=medias[k,], cov=covarianzas[k,:,:], allow_singular=True)+epsilon)
    # print('res: ', res)
    res = np.log(res)
    prob += res
  return prob
    

In [94]:
k = 7
epochs = 300
redux = 20
redux_type = 1 # NOTE: comentar sobre el resultado de '1'
q = 2
#cm = cluster map, cfm = cluster feature means, dt = dataset
clusters, cm, cfm, dataset, names = kmeans(k, epochs, redux, redux_type, q)
print(dataset[0:2,])
print(dataset.shape)

Cluster: 0 

['cerebellum', 'cerebellum', 'liver', 'liver', 'kidney', 'kidney']
Cluster: 1 

['colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'colon', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver']
Cluster: 2 

['kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 

In [92]:
n_samples = np.size(dataset, axis=0)
# cm[cluster_num] -> array de arrays (puntos)
# cfm[cluster_num] -> media del cluster 'cluster_num'
# print(dataset)
means = np.zeros((k, redux))

# print('cfm.items(): ', cfm[0])

for idx, (key, value) in enumerate(cfm.items()):
  means[idx,] += value

covarianzas = np.zeros((k, redux, redux))

# print('cm.item(): ', cm[0])

for idx, (key, value) in enumerate(cm.items()):
  # print('calculando covarianza...')
  cova = np.cov(value, rowvar=False)
  # print('cov shape: ', cova.shape)
  # print('cov: ', cova)
  covarianzas[idx,:,:] += cova # NOTE: acabo de intentar esto (abs)

pis = np.full((k,), 1/k, dtype='float64')

print('pis: ', pis)
# print('covarianzas: ', covarianzas)
print('means: ', means)

likelihood = log_likelihood(k, means, covarianzas, n_samples, pis)
prev_likelihood = 1000

print('likelihood: ', likelihood)

gammas = np.array([])
N = np.array([])

threshold = 0.000001

i = 0

while (True):
  # if abs(prev_likelihood-likelihood) < threshold:
  if prev_likelihood == likelihood:
    print('diff likelihood: ', abs(prev_likelihood-likelihood))
    break
  prev_likelihood = likelihood
  print('N: ', N)
  # TODO: revisar si estas variables de actualizan o no
  gammas = new_responsibilities(pis, means, covarianzas, n_samples, k)
  N = new_N(gammas, k, n_samples)
  means = new_medias(gammas, N, n_samples, redux, k)
  covarianzas = new_covarianzas(gammas, means, N, n_samples, redux, k)
  pis = new_pis(N, n_samples, k)
  # print('gammas: ', gammas)
  # print('k: ', k)
  # print('means: ', means)
  # print('covarianzas: ', covarianzas)
  # print('n_samples: ', n_samples)
  # print('pis: ', pis)
  likelihood = log_likelihood(k, means, covarianzas, n_samples, pis)
  print('likelihood: ', likelihood)
  i += 1

print('final likelihood: ', likelihood)

pis:  [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
means:  [[-126.56868528 -149.2535791  -137.60165527   22.01701278  -26.15068852
   -62.00871507  -21.14334221    4.10429168    2.493903     -5.35016006]
 [ 242.21393962   32.52003806   53.34063548  133.84744816  -49.16640154
    -2.70021218   -4.79861977   -5.23714772   18.69728698    1.65385659]
 [ -80.764433    -16.50981299   22.7468173    -3.86777301   43.96616355
    64.80324034  -53.28307774  -35.59198966    5.40896303   -4.28802911]
 [-113.51502314 -111.9446045   129.09070813  -21.12102157   38.07832988
    12.29904356   70.21580507   21.44885281  -17.65759117    5.14330077]
 [ 274.80396767   45.65028669  -59.68983449 -115.73223201   34.20866328
     2.17874471    2.23777398   11.49800163   -5.73933824   -1.67528766]
 [-196.16976587  199.53767183   -7.88667114  -15.14343435  -40.93606665
   -14.57210136    6.77146068    3.77799126   -3.2032236     4.51631946]
 [   0.            0.            0. 

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/Users/gabrielspranger/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:18: RuntimeWarning: Degrees of freedom <= 0 for slice
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/lib/function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/lib/function_base.py:2480: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


ValueError: array must not contain infs or NaNs

In [ ]:
# SECTION: intento fallido de GMM

# agrupamiento = {}

# for i in range(n_samples):
#   max_prob = -1
#   cluster = -1
#   for j in range(k):
#     z = gammas[i, j]
#     if z > max_prob:
#       max_prob = z
#       cluster = j
#   if agrupamiento.get(cluster) == None:
#     agrupamiento[cluster] = [dataset[i,]]
#   else:
#     agrupamiento[cluster].append(dataset[i,])

# # TODO: confirmar que cada punto dentro del cluster
# # TODO: terminar de programar como si ya estuviera
# # NOTE: con la ayuda de 'names'

# print(gammas)

# print(agrupamiento.keys())
# # print(len(agrupamiento[0]))
# print(agrupamiento)




In [95]:
# SECTION: con librerías

from sklearn import mixture
import time as clock
from collections import Counter

borrarcomillas = lambda x: x.replace('"', '')
names = np.genfromtxt('clase.csv', dtype=None, delimiter=",", skip_header=1, usecols=1, encoding='utf-8', converters={1: borrarcomillas})
# print(names)

# TODO: experimentar con 'covariance_type' e 'init_params'

labels = np.unique(names)

covariance_types = ['full', 'tied', 'diag', 'spherical']
init_parameters = ['random', 'kmeans']
tiempo_i = 0
tiempo_f = 0

def get_cluster_accuracy(gmm, labels, cov_type, t):
  # print('(tejido_predecido, tejido_verdadero)')
  agrupacion = {}
  probabilidad_conjunta = gmm.score(dataset, names)
  for idx in range(n_samples):
    prediction = gmm.predict(np.array([dataset[idx,]]))[0,]
    # tejido_predecido = labels[prediction,]
    tejido_predecido = prediction
    tejido_verdadero = names[idx,]
    # print(tejido_predecido, tejido_verdadero)
    if agrupacion.get(tejido_predecido) == None:
      agrupacion[tejido_predecido] = [tejido_verdadero]
    else:
      agrupacion[tejido_predecido].append(tejido_verdadero)

  cluster_names, occurrences, cluster_sizes = [], [], []
  
  for idx, (key, value) in enumerate(agrupacion.items()):
    print('cluster ' + str(idx) + ':\n', value)
    counter = Counter(value)
    mas_comunes = counter.most_common(1)
    print('valor más común: ', mas_comunes[0][0])
    print('#ocurrencias valor más común: ', mas_comunes[0][1])
    cluster_names.append(mas_comunes[0][0])
    occurrences.append(mas_comunes[0][1])
    cluster_sizes.append(len(value))
    print('cluster size: ', len(value))
    print('=================')
  print('log likelihood: ', probabilidad_conjunta)

  no_identificados = set(labels) - set(cluster_names)
  no_identificados_str = ''
  
  for idx, no_identificado in enumerate(no_identificados):
    if idx != len(no_identificados)-1:
      no_identificados_str += str(no_identificado) + ', '
    else:
      no_identificados_str += str(no_identificado)
  
  cluster_quantity = '7'

  precision_cluster = ''
  for idx in range(len(cluster_sizes)):
    # cluster_names[idx]: round((occurrences[idx]/cluster_sizes[idx])*100, 2)\\%\\newline
    precision_cluster += cluster_names[idx] + ': ' + str(round((occurrences[idx]/cluster_sizes[idx])*100, 2)) + '\\%\\newline '

  return cov_type + ' & ' + str(round(t, 4)) + ' & ' + no_identificados_str + ' & ' + cluster_quantity + ' & ' + precision_cluster + ' & ' + str(round(probabilidad_conjunta, 2)) + '\\\\'

for c in covariance_types:
  gmm = mixture.GaussianMixture(n_components=k, covariance_type=c, init_params='kmeans')
  tiempo_i = clock.time()
  gmm.fit(dataset, names)
  tiempo_f = clock.time()
  tiempo_total = tiempo_f - tiempo_i
  f = open('testing.txt', 'a')
  f.write(get_cluster_accuracy(gmm, labels, c, tiempo_total) + '\n\\hline\n')
  f.close()

# gmm = mixture.GaussianMixture(n_components=k, covariance_type='spherical', init_params='kmeans')

# gmm.fit(dataset)

# print(gmm.get_params())

# print(get_cluster_accuracy(gmm, labels, 'spherical', 10))




cluster 0:
 ['kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney']
valor más común:  kidney
#ocurrencias valor más común:  19
cluster size:  19
cluster 1:
 ['kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney', 'kidney']
valor más común:  kidney
#ocurrencias valor más común:  20
cluster size:  20
cluster 2:
 ['hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus', 'hippocampus